In [2]:
!pip3 install torch==1.0.1 torchvision==0.2.2 -f https://download.pytorch.org/whl/cpu/torch_stable.html

Keyring is skipped due to an exception: 'keyring.backends'
Looking in links: https://download.pytorch.org/whl/cpu/torch_stable.html

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip3 install easydict

Keyring is skipped due to an exception: 'keyring.backends'

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
# Proprocess the data, create kg etc. 
from __future__ import absolute_import, division, print_function

import os
import pickle
import gzip

import data_utils
import utils

from utils import *
from data_utils import ChallengeDataset, ChallengeDataLoader
from knowledge_graph import KnowledgeGraph

In [5]:
def generate_labels(dataset, mode='train'):
    review_file = '{}/{}.txt'.format(DATASET_DIR[dataset], mode)
    user_articles = {}  # {uid: [aid,...], ...}
    with open(review_file, 'r') as f:
        for line in f:
            line = line.strip()
            arr = line.split('\t')
            user_idx = int(arr[0])
            article_idx = int(arr[1])
            if user_idx not in user_articles:
                user_articles[user_idx] = []
            user_articles[user_idx].append(article_idx)
    save_labels(dataset, user_articles, mode=mode)


def main(data):
    dataset_n = data

    # Create Dataset instance for dataset.
    # ========== BEGIN ========== #
    print('Load', dataset_n, 'dataset from file...')
    if not os.path.isdir(TMP_DIR[dataset_n]):
        os.makedirs(TMP_DIR[dataset_n])
    dataset = ChallengeDataset(DATASET_DIR[dataset_n])
    save_dataset(dataset_n, dataset)

    # Generate knowledge graph instance.
    # ========== BEGIN ========== #
    
    print('Create', dataset_n, 'knowledge graph from dataset...')
    dataset = load_dataset(dataset_n)
    kg = KnowledgeGraph(dataset)
    kg.compute_degrees()
    save_kg(dataset_n, kg)
    # =========== END =========== #
    
    # Genereate train/test labels.
    # ========== BEGIN ========== #
    print('Generate', dataset_n, 'train/test labels.')
    generate_labels(dataset_n, 'train')
    generate_labels(dataset_n, 'test')
    # =========== END =========== #
    

if __name__ == '__main__':
    main("challenge")

Load challenge dataset from file...
Load user of size 1000
Load article of size 71
Load word of size 2246
Load related_article of size 71
Load topic of size 55
Load product of size 64
Load topic_tag of size 24
Load product_tag of size 45
Load has_topic of size 71
Load has_product of size 71
Load also_response of size 71
Load recommended_together of size 71
Load response_together of size 71
Load has_topic_tag of size 71
Load has_product_tag of size 71
Load text of size 70294 word count= 28022586
Create word sampling rate
Create challenge knowledge graph from dataset...
Load entities...
Total 3576 nodes.
Load text...
Total 333282 text edges.
Load knowledge has_topic...
Total 566 has_topic edges.
Load knowledge has_product...
Total 720 has_product edges.
Load knowledge also_response...
Total 7334 also_response edges.
Load knowledge recommended_together...
Total 368 recommended_together edges.
Load knowledge response_together...
Total 142 response_together edges.
Load knowledge has_topic_t

In [5]:
import sys
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from transe_model import KnowledgeEmbedding

In [7]:
logger = None
convergence = 0
smooth_loss_min = 0


def train(dataset='challenge',name='train_transe_model',log_dir = './', device='cuda',seed=123,gpu='0',epochs=1, batch_size=64,lr=0.5,weight_decay=0,l2_lambda=0,max_grad_norm=5.0,embed_size=300,num_neg_samples=5,steps_per_checkpoint=200):
    data = load_dataset(dataset)
    dataloader = ChallengeDataLoader(data, batch_size)
    words_to_train = epochs * data.text.word_count + 1

    model = KnowledgeEmbedding(data, device=device,seed=123,gpu='0',epochs=1, batch_size=64,lr=0.5,weight_decay=0,l2_lambda=0,max_grad_norm=5.0,embed_size=300,num_neg_samples=5,steps_per_checkpoint=200).to(device)
     #logger.info('Parameters:' + str([i[0] for i in model.named_parameters()]))
    optimizer = optim.SGD(model.parameters(), lr=lr)
    steps = 0
    smooth_loss = 0.0

    for epoch in range(1, epochs + 1):
        dataloader.reset()
        while dataloader.has_next():
            # Set learning rate.
            #lr = lr * max(1e-4, 1.0 - dataloader.finished_word_num / float(words_to_train))
            for pg in optimizer.param_groups:
                pg['lr'] = lr

            # Get training batch.
            batch_idxs = dataloader.get_batch()
            batch_idxs = torch.from_numpy(batch_idxs).to(device)

            # Train model.
            optimizer.zero_grad()
            train_loss = model(batch_idxs)
            train_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            smooth_loss += train_loss.item() / steps_per_checkpoint

            steps += 1
            if steps % steps_per_checkpoint == 0:
                logger.info('Epoch: {:02d} | '.format(epoch) +
                            'Words: {:d}/{:d} | '.format(dataloader.finished_word_num, words_to_train) +
                            'Lr: {:.5f} | '.format(lr) +
                            'Smooth loss: {:.5f}'.format(smooth_loss))
                smooth_loss = 0.0
        
        torch.save(model.state_dict(), '{}/transe_model_sd_epoch_{}.ckpt'.format(log_dir, epoch))
        

def extract_embeddings(dataset='challenge',name='train_transe_model',log_dir='./', seed=123,gpu='0',epochs=1, batch_size=64,lr=0.5,weight_decay=0,l2_lambda=0,max_grad_norm=5.0,embed_size=300,num_neg_samples=5,steps_per_checkpoint=200):
    """Note that last entity embedding is of size [vocab_size+1, d]."""
    model_file = '{}/transe_model_sd_epoch_{}.ckpt'.format(log_dir, epochs)
    print('Load embeddings', model_file)
    state_dict = torch.load(model_file, map_location=lambda storage, loc: storage)
    embeds = {
        USER: state_dict['user.weight'].cpu().data.numpy()[:-1],  # Must remove last dummy 'user' with 0 embed.
        ARTICLE: state_dict['article.weight'].cpu().data.numpy()[:-1],
        WORD: state_dict['word.weight'].cpu().data.numpy()[:-1],
        TOPIC: state_dict['topic.weight'].cpu().data.numpy()[:-1],
        PRODUCT: state_dict['product.weight'].cpu().data.numpy()[:-1],
        RARTICLE: state_dict['related_article.weight'].cpu().data.numpy()[:-1],
        TOPIC_TAG: state_dict['topic_tag.weight'].cpu().data.numpy()[:-1],
        PRODUCT_TAG: state_dict['product_tag.weight'].cpu().data.numpy()[:-1],
        
        RECOMMENDED: (
            state_dict['recommended'].cpu().data.numpy()[0],
            state_dict['recommended_bias.weight'].cpu().data.numpy()
        ),
        WITHIN: (
            state_dict['within'].cpu().data.numpy()[0],
            state_dict['within_bias.weight'].cpu().data.numpy()
        ),
        HAS_TOPIC: (
            state_dict['has_topic'].cpu().data.numpy()[0],
            state_dict['has_topic_bias.weight'].cpu().data.numpy()
        ),
        HAS_PRODUCT: (
            state_dict['has_product'].cpu().data.numpy()[0],
            state_dict['has_product_bias.weight'].cpu().data.numpy()
        ),
        HAS_TOPIC_TAG: (
            state_dict['has_topic_tag'].cpu().data.numpy()[0],
            state_dict['has_topic_tag_bias.weight'].cpu().data.numpy()
        ),
        HAS_PRODUCT_TAG: (
            state_dict['has_product_tag'].cpu().data.numpy()[0],
            state_dict['has_product_tag_bias.weight'].cpu().data.numpy()
        ),
        ALSO_RESPONSE: (
            state_dict['also_response'].cpu().data.numpy()[0],
            state_dict['also_response_bias.weight'].cpu().data.numpy()
        ),
        RECOMMENDED_TOGETHER: (
            state_dict['recommended_together'].cpu().data.numpy()[0],
            state_dict['recommended_together_bias.weight'].cpu().data.numpy()
        ),
        RESPONSE_TOGETHER: (
            state_dict['response_together'].cpu().data.numpy()[0],
            state_dict['response_together_bias.weight'].cpu().data.numpy()
        ),
    }
    save_embed(dataset, embeds)


def main(dataset='challenge',name='train_transe_model',seed=123,gpu='0',epochs=1, batch_size=64,lr=0.5,weight_decay=0,l2_lambda=0,max_grad_norm=5.0,embed_size=300,num_neg_samples=5,steps_per_checkpoint=200):

    os.environ['CUDA_VISIBLE_DEVICES'] = '1'
    device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

    log_dir = '{}/{}'.format(TMP_DIR[dataset], name)
    print(log_dir)
    if not os.path.isdir(log_dir):
        os.makedirs(log_dir)

    global logger
    logger = get_logger(log_dir + '/train_log.txt')
    set_random_seed(seed)
    train(dataset='challenge',name='train_transe_model',log_dir= '{}/{}'.format(TMP_DIR[dataset], name),device=device,seed=123,gpu='0',epochs=1, batch_size=64,lr=0.005,weight_decay=0,l2_lambda=0,max_grad_norm=5.0,embed_size=300,num_neg_samples=5,steps_per_checkpoint=200)
    extract_embeddings(dataset='challenge',name='train_transe_model',log_dir=log_dir, seed=123,gpu='0',epochs=1, batch_size=64,lr=0.005,weight_decay=0,l2_lambda=0,max_grad_norm=5.0,embed_size=300,num_neg_samples=5,steps_per_checkpoint=200)


if __name__ == '__main__':
    main(dataset='challenge',name='train_transe_model',seed=123,gpu='0',epochs=1, batch_size=64,lr=0.005,weight_decay=0,l2_lambda=0,max_grad_norm=5.0,embed_size=300,num_neg_samples=5,steps_per_checkpoint=200)

./tmp/Challenge_Dataset/train_transe_model
[INFO]  Epoch: 01 | Words: 30451/28022587 | Lr: 0.00500 | Smooth loss: 38.73018
[INFO]  Epoch: 01 | Words: 61084/28022587 | Lr: 0.00500 | Smooth loss: 37.78879
[INFO]  Epoch: 01 | Words: 90328/28022587 | Lr: 0.00500 | Smooth loss: 36.85461
[INFO]  Epoch: 01 | Words: 121101/28022587 | Lr: 0.00500 | Smooth loss: 36.04648
[INFO]  Epoch: 01 | Words: 151713/28022587 | Lr: 0.00500 | Smooth loss: 35.67448
[INFO]  Epoch: 01 | Words: 183357/28022587 | Lr: 0.00500 | Smooth loss: 35.34737
[INFO]  Epoch: 01 | Words: 213026/28022587 | Lr: 0.00500 | Smooth loss: 34.99123
[INFO]  Epoch: 01 | Words: 242182/28022587 | Lr: 0.00500 | Smooth loss: 34.03343
[INFO]  Epoch: 01 | Words: 273517/28022587 | Lr: 0.00500 | Smooth loss: 34.32667
[INFO]  Epoch: 01 | Words: 303758/28022587 | Lr: 0.00500 | Smooth loss: 34.00162
[INFO]  Epoch: 01 | Words: 333455/28022587 | Lr: 0.00500 | Smooth loss: 33.07016
[INFO]  Epoch: 01 | Words: 364129/28022587 | Lr: 0.00500 | Smooth los

In [6]:
import argparse
from collections import namedtuple
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.distributions import Categorical

from kg_env import BatchKGEnvironment

In [7]:
logger = None

SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])

class ActorCritic(nn.Module):
    def __init__(self, state_dim, act_dim, gamma=0.99, hidden_sizes=[512, 256]):
        super(ActorCritic, self).__init__()
        self.state_dim = state_dim
        self.act_dim = act_dim
        self.gamma = gamma

        self.l1 = nn.Linear(state_dim, hidden_sizes[0])
        self.l2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.actor = nn.Linear(hidden_sizes[1], act_dim)
        self.critic = nn.Linear(hidden_sizes[1], 1)

        self.saved_actions = []
        self.rewards = []
        self.entropy = []

    def forward(self, inputs):
        state, act_mask = inputs  # state: [bs, state_dim], act_mask: [bs, act_dim]
        x = self.l1(state)
        x = F.dropout(F.elu(x), p=0.5)
        out = self.l2(x)
        x = F.dropout(F.elu(out), p=0.5)

        actor_logits = self.actor(x)
        actor_logits[1 - act_mask] = -999999.0
        act_probs = F.softmax(actor_logits, dim=-1)  # Tensor of [bs, act_dim]

        state_values = self.critic(x)  # Tensor of [bs, 1]
        return act_probs, state_values

    def select_action(self, batch_state, batch_act_mask, device):
        state = torch.FloatTensor(batch_state).to(device)  # Tensor [bs, state_dim]
        act_mask = torch.ByteTensor(batch_act_mask).to(device)  # Tensor of [bs, act_dim]

        probs, value = self((state, act_mask))  # act_probs: [bs, act_dim], state_value: [bs, 1]
        m = Categorical(probs)
        acts = m.sample()  # Tensor of [bs, ], requires_grad=False
        # [CAVEAT] If sampled action is out of action_space, choose the first action in action_space.
        valid_idx = act_mask.gather(1, acts.view(-1, 1)).view(-1)
        acts[valid_idx == 0] = 0

        self.saved_actions.append(SavedAction(m.log_prob(acts), value))
        self.entropy.append(m.entropy())
        return acts.cpu().numpy().tolist()

    def update(self, optimizer, device, ent_weight):
        if len(self.rewards) <= 0:
            del self.rewards[:]
            del self.saved_actions[:]
            del self.entropy[:]
            return 0.0, 0.0, 0.0

        batch_rewards = np.vstack(self.rewards).T  # numpy array of [bs, #steps]
        batch_rewards = torch.FloatTensor(batch_rewards).to(device)
        num_steps = batch_rewards.shape[1]
        for i in range(1, num_steps):
            batch_rewards[:, num_steps - i - 1] += self.gamma * batch_rewards[:, num_steps - i]

        actor_loss = 0
        critic_loss = 0
        entropy_loss = 0
        for i in range(0, num_steps):
            log_prob, value = self.saved_actions[i]  # log_prob: Tensor of [bs, ], value: Tensor of [bs, 1]
            advantage = batch_rewards[:, i] - value.squeeze(1)  # Tensor of [bs, ]
            actor_loss += -log_prob * advantage.detach()  # Tensor of [bs, ]
            critic_loss += advantage.pow(2)  # Tensor of [bs, ]
            entropy_loss += -self.entropy[i]  # Tensor of [bs, ]
        actor_loss = actor_loss.mean()
        critic_loss = critic_loss.mean()
        entropy_loss = entropy_loss.mean()
        loss = actor_loss + critic_loss + ent_weight * entropy_loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del self.rewards[:]
        del self.saved_actions[:]
        del self.entropy[:]

        return loss.item(), actor_loss.item(), critic_loss.item(), entropy_loss.item()


class ACDataLoader(object):
    def __init__(self, uids, batch_size):
        self.uids = np.array(uids)
        self.num_users = len(uids)
        self.batch_size = batch_size
        self.reset()

    def reset(self):
        self._rand_perm = np.random.permutation(self.num_users)
        self._start_idx = 0
        self._has_next = True

    def has_next(self):
        return self._has_next

    def get_batch(self):
        if not self._has_next:
            return None
        # Multiple users per batch
        end_idx = min(self._start_idx + self.batch_size, self.num_users)
        batch_idx = self._rand_perm[self._start_idx:end_idx]
        batch_uids = self.uids[batch_idx]
        self._has_next = self._has_next and end_idx < self.num_users
        self._start_idx = end_idx
        return batch_uids.tolist()


def train( dataset='challenge',name='train_agent',seed=123,gpu='0',epochs=1, batch_size=32,lr=1e-4,max_acts=250,max_path_len=3,
    gamma=0.99,ent_weight=1e-3,act_dropout=0.5,state_history=1,hidden=[512, 256],device='cuda',log_dir='./'):
    env = BatchKGEnvironment(dataset, max_acts, max_path_len=max_path_len, state_history=state_history)
    uids = list(env.kg(USER).keys())
    dataloader = ACDataLoader(uids, batch_size)
    model = ActorCritic(env.state_dim, env.act_dim, gamma=gamma, hidden_sizes=hidden).to(device)
    logger.info('Parameters:' + str([i[0] for i in model.named_parameters()]))
    optimizer = optim.Adam(model.parameters(), lr=lr)

    total_losses, total_plosses, total_vlosses, total_entropy, total_rewards = [], [], [], [], []
    step = 0
    model.train()
    for epoch in range(1, epochs + 1):
        ### Start epoch ###
        dataloader.reset()
        while dataloader.has_next():
            batch_uids = dataloader.get_batch()
            ### Start batch episodes ###
            batch_state = env.reset(batch_uids)  # numpy array of [bs, state_dim]
            done = False
            while not done:
                batch_act_mask = env.batch_action_mask(dropout=act_dropout)  # numpy array of size [bs, act_dim]
                batch_act_idx = model.select_action(batch_state, batch_act_mask, device)  # int
                batch_state, batch_reward, done = env.batch_step(batch_act_idx)
                model.rewards.append(batch_reward)
            ### End of episodes ###

            lr_e = lr * max(1e-4, 1.0 - float(step) / (epochs * len(uids) / batch_size))
            for pg in optimizer.param_groups:
                pg['lr'] = lr_e

            # Update policy
            total_rewards.append(np.sum(model.rewards))
            loss, ploss, vloss, eloss = model.update(optimizer, device, ent_weight)
            total_losses.append(loss)
            total_plosses.append(ploss)
            total_vlosses.append(vloss)
            total_entropy.append(eloss)
            step += 1

            # Report performance
            if step > 0 and step % 100 == 0:
                avg_reward = np.mean(total_rewards) / batch_size
                avg_loss = np.mean(total_losses)
                avg_ploss = np.mean(total_plosses)
                avg_vloss = np.mean(total_vlosses)
                avg_entropy = np.mean(total_entropy)
                total_losses, total_plosses, total_vlosses, total_entropy, total_rewards = [], [], [], [], []
                logger.info(
                        'epoch/step={:d}/{:d}'.format(epoch, step) +
                        ' | loss={:.5f}'.format(avg_loss) +
                        ' | ploss={:.5f}'.format(avg_ploss) +
                        ' | vloss={:.5f}'.format(avg_vloss) +
                        ' | entropy={:.5f}'.format(avg_entropy) +
                        ' | reward={:.5f}'.format(avg_reward))
        ### END of epoch ###

        policy_file = '{}/policy_model_epoch_{}.ckpt'.format(log_dir, epoch)
        logger.info("Save model to " + policy_file)
        torch.save(model.state_dict(), policy_file)


def main(dataset='challenge',name='train_agent',seed=123,gpu='0',epochs=1, batch_size=32,lr=1e-4,max_acts=250,max_path_len=3,
    gamma=0.99,ent_weight=1e-3,act_dropout=0.5,state_history=1,hidden=[512, 256]):
    os.environ['CUDA_VISIBLE_DEVICES'] = '1'
    device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

    log_dir = '{}/{}'.format(TMP_DIR['challenge'], name)
    if not os.path.isdir(log_dir):
        os.makedirs(log_dir)

    global logger
    logger = get_logger(log_dir + '/train_log.txt')
    #logger.info(args)

    set_random_seed(seed)
    train(dataset='challenge',name='train_agent',seed=123,gpu='0',epochs=1, batch_size=32,lr=1e-4,max_acts=250,max_path_len=3,
    gamma=0.99,ent_weight=1e-3,act_dropout=0.5,state_history=1,hidden=[512, 256],device=device,log_dir=log_dir)


if __name__ == '__main__':
    main(dataset='challenge',name='train_agent',seed=123,gpu='0',epochs=1, batch_size=32,lr=1e-4,max_acts=250,max_path_len=3,
    gamma=0.99,ent_weight=1e-3,act_dropout=0.5,state_history=1,hidden=[512, 256])

Load embedding: ./tmp/Challenge_Dataset/transe_embed.pkl
[INFO]  Parameters:['l1.weight', 'l1.bias', 'l2.weight', 'l2.bias', 'actor.weight', 'actor.bias', 'critic.weight', 'critic.bias']
[INFO]  Save model to ./tmp/Challenge_Dataset/train_agent/policy_model_epoch_1.ckpt


In [8]:
import argparse
from math import log
from datetime import datetime
from tqdm import tqdm
from collections import namedtuple
import torch.optim as optim
from torch.autograd import Variable
from torch.distributions import Categorical
import threading
from functools import reduce

In [9]:
def evaluate(topk_matches, test_user_articles):
    """Compute metrics for predicted recommendations.
    Args:
        topk_matches: a list or dict of product ids in ascending order.
    """
    invalid_users = []
    # Compute metrics
    precisions, recalls, ndcgs, hits, map_scores = [], [], [], [], []
    test_user_idxs = list(test_user_articles.keys())
    pred_real_articles,pred_l,real_l=[],[],[]
    for uid in test_user_idxs:
        if uid not in topk_matches or len(topk_matches[uid]) < 10:
            invalid_users.append(uid)
            continue
        pred_list, rel_set = topk_matches[uid][::-1], test_user_articles[uid]
        #print("uid:",uid,"pred_list:",pred_list,"real_set:",rel_set)
        pred_real = "uid:"+str(uid)+' '+"pred_list:"+str(pred_list)+' '+"rel_set:"+str(rel_set)
        pred_real_articles.append(pred_real)
        pred_l.append(pred_list)
        real_l.append(rel_set)
        #print(pred_real_articles)
        if len(pred_list) == 0:
            continue
        #print("uid:",uid, "pred_list:",pred_list, "rel_set:",rel_set)
        
        dcg = 0.0
        hit_num = 0.0
        for i in range(len(pred_list)):
            if pred_list[i] in rel_set:
                dcg += 1. / (log(i + 2) / log(2))
                hit_num += 1
        # idcg
        idcg = 0.0
        for i in range(min(len(rel_set), len(pred_list))):
            idcg += 1. / (log(i + 2) / log(2))
        ndcg = dcg / idcg
        recall = hit_num / len(rel_set)
        precision = hit_num / len(pred_list)
        hit = 1.0 if hit_num > 0.0 else 0.0
        
        #map
        map_score = 0.0
        num_hits = 0.0
        score = 0.0
        for i,p in enumerate(pred_list):
            if p in rel_set and p not in pred_list[:i]:
                num_hits+=1.0
                score+=num_hits/(i+1.0)
        map_score = score/min(len(rel_set),10)
        
        ndcgs.append(ndcg)
        recalls.append(recall)
        precisions.append(precision)
        hits.append(hit)
        map_scores.append(map_score)
    
    with open(TMP_DIR['challenge'] + '/' +'pred_real_article.dat','wb+') as file:
        pickle.dump(pred_real_articles,file)
        
    with open(TMP_DIR['challenge'] + '/' +'pred_list.dat','wb+') as file:
        pickle.dump(pred_l,file)
        
    with open(TMP_DIR['challenge'] + '/' +'real_list.dat','wb+') as file:
        pickle.dump(real_l,file)
    
    avg_precision = np.mean(precisions) * 100
    avg_recall = np.mean(recalls) * 100
    avg_ndcg = np.mean(ndcgs) * 100
    avg_hit = np.mean(hits) * 100
    avg_map = np.mean(map_scores) * 100
    
    tmp = 'map: '+str(avg_map)+' '+'ndcg: '+str(avg_ndcg)+ ' '+'recall: '+str(avg_recall)+' '+'precision: '+str(avg_precision)+' '+str(len(invalid_users))
    pickle.dump(tmp, open(log_dir + '/result.txt', 'wb'))
    
    print('MAP={:.3f} | NDCG={:.3f} |  Recall={:.3f} | Precision={:.3f}'.format(
            avg_map, avg_ndcg, avg_recall, avg_precision))


def batch_beam_search(env, model, uids, device, topk=[25, 5, 1]):
    def _batch_acts_to_masks(batch_acts):
        batch_masks = []
        for acts in batch_acts:
            num_acts = len(acts)
            act_mask = np.zeros(model.act_dim, dtype=np.uint8)
            act_mask[:num_acts] = 1
            batch_masks.append(act_mask)
        return np.vstack(batch_masks)

    state_pool = env.reset(uids)  # numpy of [bs, dim]
    path_pool = env._batch_path  # list of list, size=bs
    probs_pool = [[] for _ in uids]
    model.eval()
    for hop in range(3):
        state_tensor = torch.FloatTensor(state_pool).to(device)
        acts_pool = env._batch_get_actions(path_pool, False)  # list of list, size=bs
        actmask_pool = _batch_acts_to_masks(acts_pool)  # numpy of [bs, dim]
        actmask_tensor = torch.ByteTensor(actmask_pool).to(device)
        probs, _ = model((state_tensor, actmask_tensor))  # Tensor of [bs, act_dim]
        probs = probs + actmask_tensor.float()  # In order to differ from masked actions
        topk_probs, topk_idxs = torch.topk(probs, topk[hop], dim=1)  # LongTensor of [bs, k]
        topk_idxs = topk_idxs.detach().cpu().numpy()
        topk_probs = topk_probs.detach().cpu().numpy()

        new_path_pool, new_probs_pool = [], []
        for row in range(topk_idxs.shape[0]):
            path = path_pool[row]
            probs = probs_pool[row]
            for idx, p in zip(topk_idxs[row], topk_probs[row]):
                if idx >= len(acts_pool[row]):  # act idx is invalid
                    continue
                relation, next_node_id = acts_pool[row][idx]  # (relation, next_node_id)
                if relation == SELF_LOOP:
                    next_node_type = path[-1][1]
                else:
                    next_node_type = KG_RELATION[path[-1][1]][relation]
                new_path = path + [(relation, next_node_type, next_node_id)]
                new_path_pool.append(new_path)
                new_probs_pool.append(probs + [p])
        path_pool = new_path_pool
        probs_pool = new_probs_pool
        if hop < 2:
            state_pool = env._batch_get_state(path_pool)

    return path_pool, probs_pool


def predict_paths(policy_file, path_file, dataset='challenge',name='train_agent',log_dir='./', device='cuda', seed=123,gpu='0',epochs=1,max_acts=250,max_path_len=5,
         gamma=0.99,state_history=1,hidden=[512,256],add_articles=False,topk=[25, 5, 1],run_path=True,run_eval=True):
    print('Predicting paths...')
    env = BatchKGEnvironment(dataset, max_acts, max_path_len=max_path_len, state_history=state_history)
    pretrain_sd = torch.load(policy_file)
    #print(env.state_dim,env.act_dim)
    model = ActorCritic(env.state_dim, env.act_dim, gamma=gamma, hidden_sizes=hidden).to(device)
    model_sd = model.state_dict()
    model_sd.update(pretrain_sd)
    model.load_state_dict(model_sd)

    test_labels = load_labels(dataset, 'test')
    test_uids = list(test_labels.keys())

    batch_size = 16
    start_idx = 0
    all_paths, all_probs = [], []
    pbar = tqdm(total=len(test_uids))
    while start_idx < len(test_uids):
        end_idx = min(start_idx + batch_size, len(test_uids))
        batch_uids = test_uids[start_idx:end_idx]
        paths, probs = batch_beam_search(env, model, batch_uids, device, topk=topk)
        all_paths.extend(paths)
        all_probs.extend(probs)
        start_idx = end_idx
        pbar.update(batch_size)
    predicts = {'paths': all_paths, 'probs': all_probs}
    pickle.dump(predicts, open(path_file, 'wb'))


def evaluate_paths(path_file, train_labels, test_labels, add_articles=False):
    embeds = load_embed('challenge')
    user_embeds = embeds[USER]
    response_embeds = embeds[ARTICLE][0]
    article_embeds = embeds[ARTICLE]
    scores = np.dot(user_embeds + response_embeds, article_embeds.T)

    # 1) Get all valid paths for each user, compute path score and path probability.
    results = pickle.load(open(path_file, 'rb'))
    #print("result_path:",results['paths'])
    pred_paths = {uid: {} for uid in test_labels}
    for path, probs in zip(results['paths'], results['probs']):
        if path[-1][1] != ARTICLE:
            continue
        uid = path[0][2]
        if uid not in pred_paths:
            continue
        aid = path[-1][2]
        if aid not in pred_paths[uid]:
            pred_paths[uid][aid] = []
        path_score = scores[uid][aid]
        path_prob = reduce(lambda x, y: x * y, probs)
        pred_paths[uid][aid].append((path_score, path_prob, path))
    
    # 2) Pick best path for each user-product pair, also remove pid if it is in train set.
    best_pred_paths = {}
    for uid in pred_paths:
        if uid in train_labels:
            train_aids = set(train_labels[uid])
            best_pred_paths[uid] = []
            for aid in pred_paths[uid]:
                if aid in train_aids:
                    continue
                # Get the path with highest probability
                #print("pred_path:",pred_paths)
                sorted_path = sorted(pred_paths[uid][aid], key=lambda x: x[1], reverse=True)
                best_pred_paths[uid].append(sorted_path[0])
    #print("best_pred_path:",best_pred_paths)
    
    with open(TMP_DIR['challenge'] + '/' +'best_pred_path.dat','wb+') as file:
        pickle.dump(best_pred_paths,file)
    
    # 3) Compute top 10 recommended articls for each user.
    sort_by = 'score'
    pred_labels = {}
    for uid in best_pred_paths:
        if sort_by == 'score':
            sorted_path = sorted(best_pred_paths[uid], key=lambda x: (x[0], x[1]), reverse=True)
        elif sort_by == 'prob':
            sorted_path = sorted(best_pred_paths[uid], key=lambda x: (x[1], x[0]), reverse=True)
        top10_aids = [p[-1][2] for _, _, p in sorted_path[:10]]  # from largest to smallest
        # add up to 10 pids if not enough
        if add_articles and len(top10_aids) < 10:
            train_aids = set(train_labels[uid])
            cand_aids = np.argsort(scores[uid])
            for cand_aid in cand_aids[::-1]:
                if cand_aid in train_aids or cand_aid in top10_aids:
                    continue
                top10_aids.append(cand_aid)
                if len(top10_aids) >= 10:
                    break
        # end of add
        pred_labels[uid] = top10_aids[::-1]  # change order to from smallest to largest!
    evaluate(pred_labels, test_labels)


def test(dataset='challenge',name='train_agent',log_dir='./', device='cuda', seed=123,gpu='0',epochs=1,max_acts=250,max_path_len=5,
         gamma=0.99,state_history=1,hidden=[512,256],add_articles=False,topk=[25, 5, 1],run_path=True,run_eval=True):
    
    policy_file = log_dir + '/policy_model_epoch_{}.ckpt'.format(epochs)
    path_file = log_dir + '/policy_paths_epoch{}.pkl'.format(epochs)

    train_labels = load_labels(dataset, 'train')
    test_labels = load_labels(dataset, 'test')
    
    if run_path:
        predict_paths(policy_file, path_file, dataset='challenge',name='train_agent',log_dir='./', device='cuda', seed=123,gpu='0',epochs=1,max_acts=250,max_path_len=5,
         gamma=0.99,state_history=1,hidden=[512,256],add_articles=False,topk=[25, 5, 1],run_path=True,run_eval=True)
    if run_eval:
        evaluate_paths(path_file, train_labels, test_labels)


if __name__ == '__main__':
    boolean = lambda x: (str(x).lower() == 'true')
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

    log_dir = TMP_DIR['challenge'] + '/' + 'train_agent'
    test(dataset='challenge',name='train_agent',log_dir=log_dir, device=device, seed=123,gpu='0',epochs=1,max_acts=250,max_path_len=5,
         gamma=0.99,state_history=1,hidden=[512,256],add_articles=False,topk=[25, 5, 1],run_path=True,run_eval=True)

Predicting paths...
Load embedding: ./tmp/Challenge_Dataset/transe_embed.pkl


1008it [00:47, 25.36it/s]                         

Load embedding: ./tmp/Challenge_Dataset/transe_embed.pkl
MAP=19.533 | NDCG=35.850 |  Recall=35.464 | Precision=35.464 | Invalid users=257


1008it [01:00, 25.36it/s]